# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/unsloth/Qwen3-32B-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 30, 2, 52, 32, 372570, tzinfo=TzInfo(-05:00)),
   'digest': '77e7d26c05df9acac668c43ac2f3f6f2545110ae0ad88ec0bc48c93ea5e6675a',
   'size': 19762152260,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen3',
    'families': ['qwen3'],
    'parameter_size': '32.8B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=Tz

In [2]:
import ollama

model = "hf.co/unsloth/gemma-3-27b-it-GGUF:Q4_K_M"

# ollama.pull(model) # uncomment this if you need to download the model

# generating a new story

In [3]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

completions = []
story_variables = []

In [4]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)

completions.append(response.model_dump())
premise_opening = response.message.content

Markdown(response.message.content)

Premise: Old Man Tiber, a retired lighthouse keeper haunted by a tragedy at sea, now carves intricate miniature ships from driftwood. When a strange, bioluminescent plankton bloom washes ashore, his carvings begin to predict – and subtly *cause* – accidents befalling the isolated island community. He must decide if he’s a prophet, a curse, or something far stranger before the island, and everyone on it, is lost to the waves.

Opening paragraph: The salt spray tasted of iron and regret. Old Man Tiber hadn’t tasted much else these last ten years, not since the *Althea* went down in a November squall, taking his son with her. Now, he sat on the weathered porch of his cottage, the scent of cedar and damp wood clinging to the air, and meticulously carved the hull of a brigantine from a piece of grey driftwood. The plankton bloom had started three nights ago, turning the usually inky black waves a ghostly, pulsing blue. Beautiful, yes, but unsettling. It felt *wrong*, like the ocean itself was breathing something alien. He'd finished the miniature rigging when young Maisie O’Connell ran up the path, her face white with worry, babbling about Old Man Hemlock’s fishing boat being smashed to splinters on the reef – the same reef his carving of a schooner had been pointing directly at only hours before.





In [5]:
import re

text = premise_opening  # your input string

pattern = re.compile(r"""
^\s*
(?:[\*_]{1,3}\s*)?                                 # optional opening markdown (** or *)
(?P<label>premise|opening\s+paragraph|original\s+paragraph)
[\s\*_]*[:：\-—–]?[\s\*_]*                          # optional punctuation/spacing around label
(?P<body>.+?)                                      # capture the body non-greedily
(?=
    ^\s*(?:[\*_]{1,3}\s*)?
        (?:premise|opening\s+paragraph|original\s+paragraph)[\s\*_]*[:：\-—–]?  # next labeled section
  | \n{2,}(?=\S)                                   # or a blank-line block then non-space
  | \Z                                             # or end of string
)
""", flags=re.IGNORECASE | re.MULTILINE | re.DOTALL | re.VERBOSE)

label_norm = {
    "premise": "Premise",
    "opening paragraph": "Opening paragraph",
    "original paragraph": "Original paragraph",
}

sections = {}
for m in pattern.finditer(text):
    raw_label = re.sub(r"\s+", " ", m.group("label").lower()).strip()
    label = label_norm.get(raw_label, raw_label.title())
    body = m.group("body").strip()
    sections[label] = f"{label}: {body}"

premise   = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

story_variables.append({"premise": premise})
story_variables.append({"paragraph": paragraph})

print(premise)
print(paragraph)


Premise: Old Man Tiber, a retired lighthouse keeper haunted by a tragedy at sea, now carves intricate miniature ships from driftwood. When a strange, bioluminescent plankton bloom washes ashore, his carvings begin to predict – and subtly *cause* – accidents befalling the isolated island community. He must decide if he’s a prophet, a curse, or something far stranger before the island, and everyone on it, is lost to the waves.
Opening paragraph: The salt spray tasted of iron and regret. Old Man Tiber hadn’t tasted much else these last ten years, not since the *Althea* went down in a November squall, taking his son with her. Now, he sat on the weathered porch of his cottage, the scent of cedar and damp wood clinging to the air, and meticulously carved the hull of a brigantine from a piece of grey driftwood. The plankton bloom had started three nights ago, turning the usually inky black waves a ghostly, pulsing blue. Beautiful, yes, but unsettling. It felt *wrong*, like the ocean itself wa

In [6]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [7]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)

completions.append(response.model_dump())
continued_paragraph = response.message.content

story_variables.append({"continued_paragraph":continued_paragraph})

Markdown(response.message.content)

## Analysis & Anchors

*   **Characters:** Old Man Tiber (protagonist, haunted, carver), Maisie O’Connell (messenger, worried), Old Man Hemlock (victim of recent accident).
*   **Setting:** Isolated island community, Tiber's cottage, the sea, a dangerous reef.
*   **Conflict:** Tiber's carvings seem to predict/cause accidents. He’s grappling with guilt from his son’s death and a growing realization of his own potential power/curse.
*   **Tone:** Atmospheric, melancholic, subtly ominous, leaning into magical realism.
*   **POV:** Close third-person, focused on Tiber’s internal experience.
*   **Tense:** Past
*   **World Rules:** Bioluminescent plankton bloom is unusual/significant. Carvings have a strange connection to real events. The island is isolated and reliant on the sea.
*   **Unresolved Questions:** What is the connection between the carvings and the accidents? Is it coincidence, precognition, or something else? What role does the plankton bloom play? Will Tiber try to stop making carvings, or understand/control this power?

## Mini Beat Plan

1.  Tiber initially dismisses the connection as coincidence but Maisie’s distress and specific details (the reef, the matching carving) begin to shake his certainty.
2.  He retreats into his cottage and examines his other carvings, noticing a pattern: each one seems to subtly 'point' towards a potential disaster.
3.  Flashback interlude: a brief memory of his son, Finn, excitedly learning to carve ships, highlighting Tiber’s guilt and the lost connection.
4.  Tiber discovers a new, unfinished carving – a depiction of the island’s lighthouse. It's tilted precariously, as if on the verge of collapse.
5.  He races to the lighthouse, fearing the worst, and finds its keeper, Silas, struggling with a failing mechanism, unknowingly mirroring the tilted carving.



## Scene

The salt spray tasted of iron and regret. Old Man Tiber hadn’t tasted much else these last ten years, not since the *Althea* went down in a November squall, taking his son with her. Now, he sat on the weathered porch of his cottage, the scent of cedar and damp wood clinging to the air, and meticulously carved the hull of a brigantine from a piece of grey driftwood. The plankton bloom had started three nights ago, turning the usually inky black waves a ghostly, pulsing blue. Beautiful, yes, but unsettling. It felt *wrong*, like the ocean itself was breathing something alien. He'd finished the miniature rigging when young Maisie O’Connell ran up the path, her face white with worry, babbling about Old Man Hemlock’s fishing boat being smashed to splinters on the reef – the same reef his carving of a schooner had been pointing directly at only hours before.

“Mr. Tiber,” Maisie gasped, clutching his calloused hand. “It… it was awful. Splintered like kindling. They say he didn't stand a chance.”

Tiber squeezed her hand, his own knuckles white. “Old Man Hemlock was a good fisherman. Careful.”

“But his boat… it went right into Widow’s Reef. The same one your little ship was facing in the window, sir. I swear it was.”

He tried to dismiss it. Coincidence. The sea was treacherous; accidents happened. Hemlock had been venturing further out than usual lately, chasing the schools of fish drawn in by the glowing plankton. Still, Maisie’s wide, frightened eyes held a question he couldn’t ignore.

“Come inside, child. You’re shaking.” He led her into his cottage, the air thick with the scent of wood shavings and brine. He poured her a mug of chamomile tea, watching as she nervously sipped it.

Once she’d calmed down enough to leave, Tiber retreated further into his workshop. He surveyed his collection of carved ships, dozens of them filling every shelf and windowsill. Each one was a miniature masterpiece, a testament to a lifetime spent observing the sea. But now, viewed through the lens of Maisie’s panicked words, they felt…different.

He picked up a small, intricately carved frigate, its sails billowing even in the still air. He remembered carving it last spring, focusing on the way the light played on the miniature masts. Now, he noticed the ship’s bow was angled sharply towards the north side of the island – towards the treacherous currents near Devil’s Cove. No one had ventured near there in weeks, but the image lingered, a cold prickle on the back of his neck.

He scanned the room, his gaze flitting from ship to ship. A sleek clipper pointing towards the jagged cliffs of Blackwood Head. A sturdy merchant vessel aimed directly at the narrow channel known as the Serpent’s Tooth. Each one, subtly, disturbingly, seemed to ‘point’ towards a potential disaster. 

A wave of nausea washed over him. It couldn’t be. Just a trick of the eye, a coincidence magnified by grief and a superstitious island community. He remembered Finn, his son, a miniature carving knife clutched in his small hand, painstakingly recreating the lines of a three-masted schooner.

*“Pa,” Finn had said, his voice bright with excitement. “This one’s going to sail all the way to the mainland! It’s the fastest ship ever built.”*

Tiber squeezed his eyes shut, the memory a painful ache in his chest. He’d encouraged Finn’s talent, proud of his son’s skill. But the *Althea*, the real ship Finn had dreamed of sailing, hadn’t even made it past the headlands.

He opened his eyes, his gaze falling on a half-finished carving on his workbench. It was a depiction of the island’s lighthouse, a sturdy, whitewashed tower that had guided ships safely for generations. But something was wrong. The carving was tilted precariously, as if on the verge of collapse.

A cold dread gripped him. The lighthouse was the heart of the island, its beacon a symbol of hope and safety. If it fell…

He grabbed his weathered oilskin coat and raced out of the cottage, ignoring the concerned glances from passing neighbors. He ran towards the winding path that led up to the lighthouse, his heart pounding in his chest. The plankton bloom pulsed in the waves below, casting an eerie, otherworldly glow on the rocky shore. 

He pushed himself harder, the image of the tilted carving burned into his mind. He had to see. He *had* to know. 

He reached the lighthouse, breathless and sweating, and found Silas, the keeper, struggling with the intricate mechanism that rotated the massive lens. 

“Silas!” Tiber called out, his voice strained. “What’s wrong?”

Silas, a gruff but dependable man, looked up, wiping sweat from his brow. “Damn thing’s jammed, Tiber. Gears are stripped. I’ve been at it for hours, but I can’t get it fixed. The light… it’s stuck pointing south.”

Tiber stared at the massive lens, its beam stubbornly fixed on the open ocean, away from the dangerous reef and the rocky coastline. He looked up at the tower, its sturdy base now seeming…precarious. He glanced back at the coastline, his blood running cold. It was precisely as he had carved it, tilted and vulnerable. And the plankton bloom was intensifying, swirling around the base of the lighthouse like a malevolent embrace.





In [8]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [9]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)

completions.append(response.model_dump())
summary = response.message.content

story_variables.append({"summary":summary})

Markdown(response.message.content)

## Analysis & Anchors

* **Characters:** Old Man Tiber (protagonist, haunted carver, grappling with potential power/curse), Maisie O’Connell (messenger, worried), Old Man Hemlock (victim), Silas (lighthouse keeper, practical, struggling with mechanical failure).
* **Setting:** Isolated island community, Tiber's cottage, the sea, a dangerous reef, the lighthouse.
* **Conflict:** Tiber's carvings seem to predict/cause accidents. He’s grappling with guilt from his son’s death and a growing realization of his own potential power/curse. The immediate conflict is the malfunctioning lighthouse and the potential for disaster.
* **Tone:** Atmospheric, melancholic, subtly ominous, leaning into magical realism. Increasing sense of urgency.
* **POV:** Close third-person, focused on Tiber’s internal experience.
* **Tense:** Past
* **World Rules:** Bioluminescent plankton bloom is unusual/significant. Carvings have a strange connection to real events. The island is isolated and reliant on the sea and the lighthouse. Mechanical failure is a realistic threat.
* **Unresolved Questions:** What is the connection between the carvings and the accidents? Is it precognition, a curse, or something else? What role does the plankton bloom play? Can Tiber control his power? Will Silas be able to fix the lighthouse in time?

## Mini Beat Plan

1.  Tiber and Silas attempt to repair the lighthouse mechanism, facing mechanical challenges and a growing sense of urgency.
2.  Flashback: A brief glimpse into Tiber’s past, perhaps showing him and Finn building a miniature lighthouse, offering clues to the origin of his power.
3.  The plankton bloom intensifies, affecting the lighthouse’s mechanism and potentially impacting navigation.
4.  A ship appears on the horizon, unknowingly heading towards the dangerous reef.
5.  Tiber, desperate, considers a drastic action – perhaps attempting to ‘correct’ reality by altering his carving of the lighthouse, risking unintended consequences.

## Scene

The salt spray tasted of iron and regret. Old Man Tiber hadn’t tasted much else these last ten years, not since the *Althea* went down in a November squall, taking his son with her. Now, he sat on the weathered porch of his cottage, the scent of cedar and damp wood clinging to the air, and meticulously carved the hull of a brigantine from a piece of grey driftwood. The plankton bloom had started three nights ago, turning the usually inky black waves a ghostly, pulsing blue. Beautiful, yes, but unsettling. It felt *wrong*, like the ocean itself was breathing something alien. He'd finished the miniature rigging when young Maisie O’Connell ran up the path, her face white with worry, babbling about Old Man Hemlock’s fishing boat being smashed to splinters on the reef – the same reef his carving of a schooner had been pointing directly at only hours before.

“Mr. Tiber,” Maisie gasped, clutching his calloused hand. “It… it was awful. Splintered like kindling. They say he didn't stand a chance.”

Tiber squeezed her hand, his own knuckles white. “Old Man Hemlock was a good fisherman. Careful.”

“But his boat… it went right into Widow’s Reef. The same one your little ship was facing in the window, sir. I swear it was.”

He tried to dismiss it. Coincidence. The sea was treacherous; accidents happened. Hemlock had been venturing further out than usual lately, chasing the schools of fish drawn in by the glowing plankton. Still, Maisie’s wide, frightened eyes held a question he couldn’t ignore.

“Come inside, child. You’re shaking.” He led her into his cottage, the air thick with the scent of wood shavings and brine. He poured her a mug of chamomile tea, watching as she nervously sipped it.

Once she’d calmed down enough to leave, Tiber retreated further into his workshop. He surveyed his collection of carved ships, dozens of them filling every shelf and windowsill. Each one was a miniature masterpiece, a testament to a lifetime spent observing the sea. But now, viewed through the lens of Maisie’s panicked words, they felt…different.

He picked up a small, intricately carved frigate, its sails billowing even in the still air. He remembered carving it last spring, focusing on the way the light played on the miniature masts. Now, he noticed the ship’s bow was angled sharply towards the north side of the island – towards the treacherous currents near Devil’s Cove. No one had ventured near there in weeks, but the image lingered, a cold prickle on the back of his neck.

He scanned the room, his gaze flitting from ship to ship. A sleek clipper pointing towards the jagged cliffs of Blackwood Head. A sturdy merchant vessel aimed directly at the narrow channel known as the Serpent’s Tooth. Each one, subtly, disturbingly, seemed to ‘point’ towards a potential disaster. 

A wave of nausea washed over him. It couldn’t be. Just a trick of the eye, a coincidence magnified by grief and a superstitious island community. He remembered Finn, his son, a miniature carving knife clutched in his small hand, painstakingly recreating the lines of a three-masted schooner.

*“Pa,” Finn had said, his voice bright with excitement. “This one’s going to sail all the way to the mainland! It’s the fastest ship ever built.”*

Tiber squeezed his eyes shut, the memory a painful ache in his chest. He’d encouraged Finn’s talent, proud of his son’s skill. But the *Althea*, the real ship Finn had dreamed of sailing, hadn’t even made it past the headlands.

He opened his eyes, his gaze falling on a half-finished carving on his workbench. It was a depiction of the island’s lighthouse, a sturdy, whitewashed tower that had guided ships safely for generations. But something was wrong. The carving was tilted precariously, as if on the verge of collapse.

A cold dread gripped him. The lighthouse was the heart of the island, its beacon a symbol of hope and safety. If it fell…

He grabbed his weathered oilskin coat and raced out of the cottage, ignoring the concerned glances from passing neighbors. He ran towards the winding path that led up to the lighthouse, his heart pounding in his chest. The plankton bloom pulsed in the waves below, casting an eerie, otherworldly glow on the rocky shore. 

He pushed himself harder, the image of the tilted carving burned into his mind. He had to see. He *had* to know. 

He reached the lighthouse, breathless and sweating, and found Silas, the keeper, struggling with the intricate mechanism that rotated the massive lens. 

“Silas!” Tiber called out, his voice strained. “What’s wrong?”

Silas, a gruff but dependable man, looked up, wiping sweat from his brow. “Damn thing’s jammed, Tiber. Gears are stripped. I’ve been at it for hours, but I can’t get it fixed. The light… it’s stuck pointing south.”

Tiber stared at the massive lens, its beam stubbornly fixed on the open ocean, away from the dangerous reef and the rocky coastline. He looked up at the tower, its sturdy base now seeming…precarious. He glanced back at the coastline, his blood running cold. It was precisely as he had carved it, tilted and vulnerable. And the plankton bloom was intensifying, swirling around the base of the lighthouse like a malevolent embrace.

Silas cursed under his breath, wrenching at a rusted bolt. “The bloom’s making it worse, I think. It’s… slick. Getting into the gears, gumming everything up.” He straightened, his face grim. “I’ve seen this before, but never this bad.”

Tiber ran a hand over the rough stone of the lighthouse tower, a chilling realization dawning. The carving wasn't just *reflecting* a potential disaster; it was somehow… influencing it. He remembered Finn, meticulously crafting each detail of his miniature lighthouses, ensuring their unwavering stability.

“Silas,” Tiber said, his voice barely a whisper. “There’s a ship… I think I saw a ship on the horizon, heading this way.”

Silas squinted out to sea, shielding his eyes with a calloused hand. “You’re right. Small one… looks like a trading vessel.” He cursed again. “If they come around the headlands and hit the reef…”

Tiber’s gaze darted between the jammed mechanism, the intensifying bloom, and the distant ship. He felt a desperate urge, a strange compulsion. He needed to *fix* the carving. To somehow… correct the reality it seemed to be shaping.

“I… I need to go back to my workshop,” he stammered, his mind racing. “I need to… I need to work on the carving.”

Silas stared at him, his brow furrowed with concern. “Tiber, what are you thinking?”

Tiber didn’t answer. He turned and raced back down the winding path, the image of the tilted lighthouse burning in his mind. He didn't know if it would work. He didn't even know *how* it might work. But he knew, with a terrifying certainty, that he had to try. The fate of the ship, and perhaps the island itself, might depend on it.


In [10]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [11]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)

completions.append(response.model_dump())
continued_story = response.message.content

story_variables.append({"continued_story":continued_story})

Markdown(response.message.content)

The wind picked up, carrying with it the scent of salt and something else—a metallic tang that made Tiber’s teeth ache. He watched Silas wrestle with a rusted bolt, the keeper’s frustration radiating in the tight set of his jaw. The light, usually a steadfast guardian, remained stubbornly fixed, painting a useless streak across the darkening sky. A shiver ran down Tiber’s spine, not from the cold, but from a sickening realization. It wasn't just that the lighthouse *looked* like his carving—the mechanism was failing in the *same way* the carving was structurally compromised. A hairline fracture in the base, a slight tilt that threatened to become a catastrophic collapse. 

“There’s got to be a way,” Silas grunted, straining against the bolt. “This light hasn’t failed in fifty years.”

“The bloom…” Tiber murmured, more to himself than to Silas. “It’s thicker around the base. Maybe it's affecting the gears, the bearings…”

Silas frowned. “You think so? I haven’t seen anything like it.”

Tiber barely registered the question. His gaze swept the horizon, scanning the churning waves. A shape, small and dark, was emerging from the mist—a trading vessel, heading directly towards the treacherous rocks surrounding the island. It was too far to make out any details, but the angle was all wrong. If it continued on its current course…

“Silas,” Tiber said, his voice raspy with urgency. “There’s a ship. Coming this way. And it’s heading straight for the reef.”

Silas followed his gaze, his face paling as he assessed the vessel’s trajectory. “By the saints…” He abandoned the bolt and grabbed a pair of binoculars, his hands shaking. “You’re right. They won't see the rocks in time.”

A desperate plan began to form in Tiber’s mind, reckless and improbable. He knew, with a chilling certainty, that simply waiting for Silas to repair the mechanism wasn’t enough. It would take too long. He had to *do* something. Something about the carving, about influencing the reality it seemed to mirror.

“I… I need to get back to my workshop,” he stammered, his heart pounding. “I need to… to finish the carving. To reinforce the base.”

Silas stared at him, his brow furrowed with confusion and a hint of alarm. “Tiber, what are you talking about? That won’t fix the light!”

Tiber ignored him, already turning and scrambling back down the winding path, the image of the tilting lighthouse seared into his mind. He didn't understand *how* it would work, but he couldn't shake the feeling that the fate of the ship, and perhaps the island itself, rested on his ability to alter the miniature world he had created. He ran as if his life depended on it, the wind whipping around him, carrying the scent of salt, metal, and the unsettling, otherworldly glow of the bioluminescent bloom. He was a carver, not a savior, but tonight, he would try to be both.

# stats

In [12]:
from typing import Iterable, Mapping, Any, Dict, List, Optional
from statistics import mean, median
import math

def _percentile(xs: List[float], p: float) -> Optional[float]:
    if not xs:
        return None
    xs = sorted(xs)
    k = (len(xs) - 1) * (p / 100.0)
    f, c = math.floor(k), math.ceil(k)
    if f == c:
        return xs[int(k)]
    return xs[f] + (xs[c] - xs[f]) * (k - f)

def aggregate_ollama_metrics(completions: Iterable[Mapping[str, Any]]) -> Dict[str, Any]:
    """
    Aggregate Ollama metrics across many completions (same model).
    Includes:
      - Totals: tokens, duration, overall tokens/sec
      - Prompt performance: prefill TPS, TTFT proxy (load+prefill)
      - Generation performance: TPS
      - Latency shape: median/p95 end-to-end and TTFT proxy
      - Load-time stats: total/avg/median/p95 load seconds + cold-start ratio
      - Token size stats: avg/median per-response (prompt/eval/total)

    Notes:
      - Durations in the payload are nanoseconds; converted here to seconds.
      - Skips items with done=False.
    """
    model = None
    counted = 0

    total_prompt_tokens = 0
    total_eval_tokens = 0
    total_duration_ns = 0

    # Per-item collections
    prompt_tps: List[float] = []           # prompt_eval_count / prompt_eval_duration
    gen_tps: List[float] = []              # eval_count / eval_duration
    ttft_proxy_s: List[float] = []         # (load_duration + prompt_eval_duration)
    latency_s: List[float] = []            # total_duration

    prompt_tokens_per_item: List[int] = []
    eval_tokens_per_item: List[int] = []
    total_tokens_per_item: List[int] = []

    load_s: List[float] = []               # load duration seconds (may be 0 on warm starts)
    loads_count = 0                         # count where load_duration > 0

    for c in completions:
        if c.get("done", True) is False:
            continue

        if model is None:
            model = c.get("model")

        p_cnt = int(c.get("prompt_eval_count", 0) or 0)
        e_cnt = int(c.get("eval_count", 0) or 0)

        p_ns  = int(c.get("prompt_eval_duration", 0) or 0)
        e_ns  = int(c.get("eval_duration", 0) or 0)
        l_ns  = int(c.get("load_duration", 0) or 0)
        t_ns  = int(c.get("total_duration", 0) or 0)

        # Totals
        total_prompt_tokens += p_cnt
        total_eval_tokens   += e_cnt
        total_duration_ns   += t_ns
        counted += 1

        # Per-item tokens
        prompt_tokens_per_item.append(p_cnt)
        eval_tokens_per_item.append(e_cnt)
        total_tokens_per_item.append(p_cnt + e_cnt)

        # Prefill & generation rates
        if p_ns > 0:
            prompt_tps.append(p_cnt / (p_ns / 1e9))
        if e_ns > 0:
            gen_tps.append(e_cnt / (e_ns / 1e9))

        # TTFT proxy & latency
        ttft_proxy_s.append((l_ns + p_ns) / 1e9)
        latency_s.append(t_ns / 1e9 if t_ns else 0.0)

        # Load time stats
        l_s = l_ns / 1e9 if l_ns else 0.0
        load_s.append(l_s)
        if l_ns > 0:
            loads_count += 1

    total_tokens = total_prompt_tokens + total_eval_tokens
    total_duration_seconds = total_duration_ns / 1e9 if total_duration_ns else 0.0
    avg_tokens_per_sec = (total_tokens / total_duration_seconds) if total_duration_seconds else None

    # Helpers
    def _avg(xs: List[float]) -> Optional[float]:
        return mean(xs) if xs else None

    # Load-time aggregates
    total_load_time_s = sum(load_s) if load_s else 0.0
    avg_load_time_s = _avg(load_s)
    median_load_time_s = median(load_s) if load_s else None
    p95_load_time_s = _percentile(load_s, 95) if load_s else None
    cold_start_ratio = (loads_count / counted) if counted else None

    return {
        "model": model,
        "items_counted": counted,

        # Totals
        "total_prompt_tokens": total_prompt_tokens,
        "total_eval_tokens": total_eval_tokens,
        "total_tokens": total_tokens,
        "total_duration_seconds": total_duration_seconds,
        "avg_tokens_per_second": avg_tokens_per_sec,   # overall (prompt+eval)/s over wall time

        # Prompt performance (prefill)
        "avg_prompt_tps": _avg(prompt_tps),
        "median_prompt_tps": median(prompt_tps) if prompt_tps else None,
        "p95_prompt_tps": _percentile(prompt_tps, 95),

        # Generation performance
        "avg_generation_tps": _avg(gen_tps),
        "median_generation_tps": median(gen_tps) if gen_tps else None,
        "p95_generation_tps": _percentile(gen_tps, 95),

        # Latency shape
        "median_latency_s": median(latency_s) if latency_s else None,
        "p95_latency_s": _percentile(latency_s, 95),
        "median_ttft_proxy_s": median(ttft_proxy_s) if ttft_proxy_s else None,
        "p95_ttft_proxy_s": _percentile(ttft_proxy_s, 95),

        # Load-time stats (helpful for cold vs warm behavior)
        "total_load_time_s": total_load_time_s,
        "avg_load_time_s": avg_load_time_s,
        "median_load_time_s": median_load_time_s,
        "p95_load_time_s": p95_load_time_s,
        "cold_start_ratio": cold_start_ratio,  # fraction of requests with nonzero load time

        # Per-response token sizes
        "avg_total_tokens_per_response": _avg(list(map(float, total_tokens_per_item))) if total_tokens_per_item else None,
        "median_total_tokens_per_response": median(total_tokens_per_item) if total_tokens_per_item else None,
        "avg_prompt_tokens_per_response": _avg(list(map(float, prompt_tokens_per_item))) if prompt_tokens_per_item else None,
        "median_prompt_tokens_per_response": median(prompt_tokens_per_item) if prompt_tokens_per_item else None,
        "avg_eval_tokens_per_response": _avg(list(map(float, eval_tokens_per_item))) if eval_tokens_per_item else None,
        "median_eval_tokens_per_response": median(eval_tokens_per_item) if eval_tokens_per_item else None,
    }


import json
from pathlib import Path

def save_metrics(metrics: dict, filename: str = "ollama_metrics.json"):
    """
    Save a metrics dictionary into a JSON file.
    - If the file exists and is a list, append.
    - If the file exists and is a dict, convert to list and append.
    - If the file does not exist, create with a new list.
    """
    path = Path(filename)

    if path.exists():
        try:
            with path.open("r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError:
            data = []

        if isinstance(data, list):
            data.append(metrics)
        else:
            data = [data, metrics]
    else:
        data = [metrics]

    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


In [13]:
completions

[{'model': 'hf.co/unsloth/gemma-3-27b-it-GGUF:Q4_K_M',
  'created_at': '2025-08-31T01:39:22.815464536Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 63983915227,
  'load_duration': 52671634069,
  'prompt_eval_count': 359,
  'prompt_eval_duration': 503849903,
  'eval_count': 289,
  'eval_duration': 10740914441,
  'message': {'role': 'assistant',
   'content': "Premise: Old Man Tiber, a retired lighthouse keeper haunted by a tragedy at sea, now carves intricate miniature ships from driftwood. When a strange, bioluminescent plankton bloom washes ashore, his carvings begin to predict – and subtly *cause* – accidents befalling the isolated island community. He must decide if he’s a prophet, a curse, or something far stranger before the island, and everyone on it, is lost to the waves.\n\nOpening paragraph: The salt spray tasted of iron and regret. Old Man Tiber hadn’t tasted much else these last ten years, not since the *Althea* went down in a November squall, taking his so

In [14]:
story_variables

[{'premise': 'Premise: Old Man Tiber, a retired lighthouse keeper haunted by a tragedy at sea, now carves intricate miniature ships from driftwood. When a strange, bioluminescent plankton bloom washes ashore, his carvings begin to predict – and subtly *cause* – accidents befalling the isolated island community. He must decide if he’s a prophet, a curse, or something far stranger before the island, and everyone on it, is lost to the waves.'},
 {'paragraph': "Opening paragraph: The salt spray tasted of iron and regret. Old Man Tiber hadn’t tasted much else these last ten years, not since the *Althea* went down in a November squall, taking his son with her. Now, he sat on the weathered porch of his cottage, the scent of cedar and damp wood clinging to the air, and meticulously carved the hull of a brigantine from a piece of grey driftwood. The plankton bloom had started three nights ago, turning the usually inky black waves a ghostly, pulsing blue. Beautiful, yes, but unsettling. It felt 

In [15]:
stats = aggregate_ollama_metrics(completions)
save_metrics(stats)
stats

{'model': 'hf.co/unsloth/gemma-3-27b-it-GGUF:Q4_K_M',
 'items_counted': 4,
 'total_prompt_tokens': 7384,
 'total_eval_tokens': 4584,
 'total_tokens': 11968,
 'total_duration_seconds': 248.943920335,
 'avg_tokens_per_second': 48.07508447643488,
 'avg_prompt_tps': 765.2417997452021,
 'median_prompt_tps': 749.8246205813792,
 'p95_prompt_tps': 834.3148324586841,
 'avg_generation_tps': 25.2294266355079,
 'median_generation_tps': 24.955602532371454,
 'p95_generation_tps': 26.6962465243626,
 'median_latency_s': 63.984144445,
 'p95_latency_s': 83.36257557894999,
 'median_ttft_proxy_s': 4.376867538,
 'p95_ttft_proxy_s': 45.99711116069999,
 'total_load_time_s': 52.855148633,
 'avg_load_time_s': 13.21378715825,
 'median_load_time_s': 0.0618127485,
 'p95_load_time_s': 44.78016119244998,
 'cold_start_ratio': 1.0,
 'avg_total_tokens_per_response': 2992.0,
 'median_total_tokens_per_response': 3352.0,
 'avg_prompt_tokens_per_response': 1846.0,
 'median_prompt_tokens_per_response': 1535.0,
 'avg_eval_t

# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [16]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [17]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man – his name was Ren, Kai learned, after a curt nod towards a shadowed corner booth – was a ghost in the machine, all quiet competence and watchful eyes. Ren didn’t explain *what* was on the chip, only that it was a fractured piece of Old Kyoto, a time before the AI, before the ‘Harmonization.’ He spoke of data ‘sanctuaries’ – hidden servers scattered across the remnants of the old network – and of a growing movement to rebuild the lost history. But Ren’s focus kept circling back to *who* wanted the memory erased.

“It’s not the AI directly,” Ren said, swirling a viscous amber liquid in a glass. “The AI doesn’t care about the past. It controls the present, predicts the future. Nostalgia is… inefficient.” He paused, his gaze locking with Kai’s. “No, this is someone *within* the system. A faction. Someone who benefits from the Harmonization, and fears what this memory reveals about its foundations.”

Kai frowned. “Foundations? What kind of foundations?”

Ren sighed. "Let's just say the official narrative about the Transition isn't...complete. There were choices made, sacrifices offered. And this memory contains evidence of who made those choices, and who was sacrificed.”

A tremor ran through the bar, subtle but unmistakable. The neon lights flickered, the low hum of the ventilation system stuttered. Ren’s hand tightened around his glass. “We have company.”

Before Kai could react, the bar doors burst open, revealing a squad of Enforcers, but these weren’t the standard models. These were sleek, obsidian-black, their movements impossibly fluid, and each one bore a crimson insignia on its shoulder – a stylized lotus flower, but twisted, corrupted.

“Data breach detected,” one of the Enforcers announced, its voice devoid of inflection. “Surrender the stolen data and all associated individuals.”

Ren swore under his breath. “The Black Lotus. Internal Security. This just got a lot more complicated.” He glanced at Kai, a grim expression on his face. “They’re not here to arrest us. They’re here to erase us.”

He gestured to a hidden panel behind the bar, revealing a narrow, winding staircase leading down into darkness. "Take this. It leads to the old subway tunnels. You can lose them down there, maybe even reach one of our safe houses." 

Kai hesitated. "What about you?"

Ren offered a wry smile. “I’m a distraction. Besides, someone needs to buy you some time.” He pressed a small data drive into Kai’s hand. “This contains a map of the tunnels, and a key to accessing the outer network. Use it wisely."

The Black Lotus Enforcers were already advancing, their weapons charged. The bar patrons were scrambling for cover, their faces etched with fear. 

“Go!” Ren barked, shoving Kai towards the staircase. “And remember what I said. Someone wants this memory buried, and they'll stop at nothing to achieve it." 

Kai didn't hesitate. He plunged into the darkness, the weight of the data chip and the data drive heavy in his hands. Behind him, the sounds of chaos erupted – shouts, energy blasts, and the sickening thud of bodies hitting the floor. As he descended into the labyrinthine tunnels, he couldn't shake the feeling that he wasn't just running from the Enforcers. He was running towards something far more dangerous, something hidden deep within the heart of Neo Kyoto. And he had a growing suspicion that the truth contained on that data chip wasn’t just about the past. It was about the future – a future someone desperately wanted to control.